## СЕРВИС ФОРМИРОВАНИЯ ЗАДАЧ ДЛЯ МОСКВИЧЕЙ ПО КОНТРОЛЮ РАБОТ ПОДРЯДЧИКОВ В СФЕРЕ ГОРОДСКОГО БЛАГОУСТРОЙСТВА
### Цель
**Разработка сервиса, который с помощью распознавания документации проектов городского благоустройства сформирует для москвичей задания по мониторингу выполненных подрядчиками работ**

## 1. Открываем смету
### 1.1. Необходимо создать функцию:
- открываем смету,
- вытаскиваем адрес,
- наименование или индекс листа - задает пользователь,
- обрабатываем от пропусков и шапок (вверху, внизу),
- переименовываем столбцы, чтобы совпадали названия в справочниках СПГЗ и КПГЗ:
    - интеграция со справочником СН и ТСН,
    - сравниваем со справочником СПГЗ и КПГЗ;
- обработка результата (см. ниже)- вывести в отдельный файл. 


## 2. Разделы и работы, в которых употребляются словосочетания ниже, не отражающие суть ключевых работ являются исключениями и не должны участвовать в дальнейшем разборе сметы: 
- Вывоз мусора 
- Погрузка мусора 
- Перевозка мусора 
- Возвратные материалы 
- Пуско-наладочные работы и т.д. 

Данные исключения сформированы в сервисе в виде справочника и могут быть отредактированы пользователем при необходимости (удалены, изменены, добавлены новые). Обратите внимание, что данный справочник является уже 5-ым на сервисе.
## 3. Результат:
- **ID**-  Из справочника, приложение 3. ID
- **КПГЗ** Выбранный пользователем или соответствующий выбранному шаблону ТЗ
- **СПГЗ** Один или несколько определенный сервисом. Може быть пустым, при этом не для каждой работы в смете должна (нужна) быть выявлена позиция СПГЗ
- **Объем** Подтягивается из сметы: Кол-во единиц
- **Единица измерения** Подтягивается из сметы: Единица измерения
- **Цена за единицу ТРУ, руб** *Рассчитываемое поле:* Стоимость за ТРУ, руб / Количество
- **Стоимость за ТРУ, руб** Подтягивается из сметы: ВСЕГО затрат, руб. При корректировке пол вручную должен быть произведен пересчет с предупреждением совпадения общей суммы по смете или возможность перераспределить остаток по всем работам в смете поровну или суммировать с выбранными позициями
- **Адрес** Вставляется при первоначальном выборе разбираемой сметы. При незаполненном поле осуществляется поиск адреса внутри сметы.

In [1]:
import os
import pandas as pd
import numpy as np
import re
from functools import reduce

pd.set_option('display.max_columns', 500)
pd.options.display.max_colwidth=250

## 1. Открываем смету

In [2]:
def estimate_read(url, sheet):
    # Загрузка сметы в гугл-документы 
    url.split('/')
    id = url.split('/')[5]
    df = pd.read_excel(f'https://docs.google.com/spreadsheets/d/{id}/export?format=xlsx', sheet_name = sheet, skiprows=9, header=None)

    # Удаляем "шабку" внизу
    df.drop(df.tail(8).index,inplace=True)

    # Удалим строки с нан
    df.dropna(axis=0, how='all', inplace=True)

    # Удалим столбцы с нан
    df.dropna(axis=1, how='all', inplace=True)

    # Вытащим адрес в новый столбец
    word = "адрес"
    mylist = df[0].tolist()

    try:
        df['address'] = next((s for s in mylist if word in s), None).split('адресу: ')[1]
    except:
        df['address'] = ''
        
    # Уберем "шабку" из анализа
    df = (
        df.loc[21:]
        .reset_index(drop=True)
    )
    try:
        df['address'] = list(df[0])[0].split('адресу: ')[1]
    except:
        df['address'] = ''
    
    df = df[[1,2,3,4,5,15, 'address']]
    # Удалим строки с нан и переименуем столбцы
    df = (
        df
        .dropna(axis=0, how='all')
        .reset_index(drop=True)
        .replace(np.nan, '')
        .rename(columns={1: 'code', 2: 'work', 3: 'count', 4: 'unit', 5: 'unit_price', 15: 'total'})
    )
# 'code'- код\ Шифр расценки и коды ресурсов
# 'work' - наименование работы 
# 'count' - единица измерения 
# 'unit' - кол-во единиц 
# 'unit_price - цена на ед. изм. руб. 
# 'total' - ВСЕГО затрат, руб.

    return df

In [3]:
# Откроем смету с определенным листом (индекс начинается с нуля)
df = estimate_read(
    'https://docs.google.com/spreadsheets/d/1UWd16ahFAVwoc1CuJErjR5kZnbMNNp7c5FRBPedaQsQ/edit?usp=sharing', 0)

print(df.shape)
df.head()

(419, 7)


,code,work,count,unit,unit_price,total,address
0,,,,,,,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
1,,,,,,,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
2,2.1-3301-2-1/1,Исправление профиля щебеночных оснований с добавлением нового материала,1000 м2,0.04665,,,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
3,,"Объем: 0,04665=(1,555*0,3)/10",,,,,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
4,,ЗП,,,15737.51,,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"


### Очистим таблицу

In [4]:
def estimate_clear(df):
    # Создадим список "бэк граунда работ"
    data1 = df['work'].value_counts().reset_index()
    data1 = data1[(data1['work'] < 5)]

    list_work = list(data1['index'])
    list_work.append('')

    # Оставим названия работ
    df = (
        df[df['work'].isin(list_work)]
        .reset_index(drop=True)
    )
    # Уберем строки с пропусками
    df = df[~(df['code'] == "") | ~(df['total'] == "")]

    # Преобразуем типы
    df['total'] = df['total'].replace('', np.nan).fillna(0).astype('float')
    df['unit'] = df['unit'].replace('', np.nan).fillna(0).astype('float')

    # Выделим две таблицы, почистим и объединим.
    df1 = (
        df
        .drop(['total'], axis=1)
        .reset_index(drop=True)
        )
    df2 = (
        df[['total']]
        .reset_index(drop=True)
    )
    # Добавим строки вниз
    df3 = pd.DataFrame({'total':[0,0,0]})
    df2 = pd.concat([df2, df3], ignore_index=True)

    # Удалим первые строки
    df2 = df2.loc[3:127].reset_index(drop=True)
    df = (
        df1
        .join(df2, how='left')
        .query('code != ""')
        .reset_index(drop=True)
    )
    return df

In [5]:
df1 = estimate_clear(df)
print(df1.shape)
df1.head()

(75, 7)


,code,work,count,unit,unit_price,address,total
0,2.1-3301-2-1/1,Исправление профиля щебеночных оснований с добавлением нового материала,1000 м2,0.046650,,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1",11767.33
1,21.1-12-35,"Щебень из естественного камня для строительных работ, марка 1200-800, фракция 10-20 мм",м3,-0.536475,1908.27,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1",0.00
2,21.1-12-29,"Щебень из естественного камня для строительных работ, марка 600-400, фракция 5-10 мм",м3,3.102225,1487.52,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1",0.00
3,2.1-3103-18-1/1,"Устройство покрытий из асфальтобетонных смесей вручную, толщина 4 см (5 см)",100 м2,1.555000,,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1",59924.59
4,21.3-3-18,"Смеси асфальтобетонные дорожные горячие мелкозернистые, марка I, тип Б",т,-14.896900,2727.65,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1",0.00


### Найдем ключевые работы по шифру или по названию. 

## 2. Разделы и работы, в которых употребляются словосочетания ниже, не отражающие суть ключевых работ являются исключениями и не должны участвовать в дальнейшем разборе сметы: 
- Вывоз мусора 
- Погрузка мусора 
- Перевозка мусора 
- Возвратные материалы 
- Пуско-наладочные работы и т.д. 

Данные исключения сформированы в сервисе в виде справочника и могут быть отредактированы пользователем при необходимости (удалены, изменены, добавлены новые). Обратите внимание, что данный справочник является уже 5-ым на сервисе.

In [6]:
def get_key_works(df, df1):
    # Загрузим единый справочник СН и ТСН
    url = 'https://docs.google.com/spreadsheets/d/1S4GHtIwV5TdGPSQ92bWkMzMSVH8WzEVbMSTB8iF4NFA/edit?usp=sharing'
    url.split('/')
    id = url.split('/')[5]
    sn_tsn = pd.read_excel(f'https://docs.google.com/spreadsheets/d/{id}/export?format=xlsx')

    #  Проведем сопоставление данных в загруженной смете и внутреннем справочнике ТСН, СН.
    df_sntsn = (
        df[['code', 'work']]
        .merge(sn_tsn[['code', 'work', 'count']], how='left', on=['code', 'work'])
    )

    # Уберем пустые строки
    df_sntsn = df_sntsn.loc[~(df_sntsn['code'] == '')].reset_index(drop=True)

    # Вытащим новую таблицу исключений
    words = ['мусор', 'возвратн', 'пуско-налад', 'свалк', 'отход']
    list = '|'.join(words)
    ignor_df = (
    df_sntsn
        .loc[df_sntsn['work'].str.contains(list, regex=True)]
        .sort_values(by='code')
        .reset_index(drop=True)
    )
    # Сохраним новый 5й справочник с работами исключениями
    ignor_df.to_excel('ignor_df.xlsx', index=False)

    # Список уникальных кодов для исключения
    ignor_code = ignor_df['code'].unique().tolist()

    # Вытащим вспомогательные работы
    words = ['азборка', 'оски', 'выравниваю']
    list = '|'.join(words)
    secondary_work = (
    df_sntsn
        .loc[df_sntsn['work'].str.contains(list, regex=True)]
        .sort_values(by='code')
        .reset_index(drop=True)
    )

    # Список уникальных кодов 
    secondary_work_list = secondary_work['code'].unique().tolist()

    # Выявим работы где появилась единица измерения, они соответствуют СПГЗ. 
    # Отсортируем таблицу списками
    
    df = (
        df_sntsn
        .query('code != @ignor_code & code != @secondary_work_list & count != 0')
        .drop_duplicates(keep='first')
        .replace(np.nan, 0)
        .reset_index(drop=True)
    )
    # Вытащим ключевые работы
    kw_l = ['стройство', 'становка' 'покрыт', 'становка', 'кладка', 'краска']
    lst = '|'.join(kw_l)
    key_works = (
        df
        .loc[df['work'].str.contains(lst, regex=True)]
        .sort_values(by='code')
        .drop_duplicates(keep='first')
        .reset_index(drop=True)
    )
    # Список уникальных кодов 
    key_works_list = key_works['code'].unique().tolist()

    # Добавим столбцы со стоимостью
    df = (
        key_works
        .merge(df1, how= 'left', on=['code', 'work'])
        .drop('count_x', axis=1)
        .rename(columns={'count_y': 'count'})
    )

    # Цена за единицу ТРУ, руб- unit_price *Рассчитываемое поле:* Стоимость за ТРУ, руб / Количество
    df['unit_price'] = round(df['total'] / df['unit'], 2)
    return df

In [7]:
df = get_key_works(df, df1)
print(df.shape)
df.head()

(12, 7)


,code,work,count,unit,unit_price,address,total
0,1.10-3403-2-1/1,"Устройство покрытий дощатых толщиной, мм 28",100 м2,2.800,0.00,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1",0.00
1,1.14-3203-14-7/1,Окраска масляными составами за два раза металлических поверхностей решеток и оград,100 м2,6.600,0.00,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1",0.00
2,2.1-3103-17-1/1,"Устройство покрытий тротуаров из бетонной плитки типа ""Брусчатка"" рядовым или паркетным мощением",100 м2,14.840,0.00,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1",0.00
3,2.1-3103-18-1/1,"Устройство покрытий из асфальтобетонных смесей вручную, толщина 4 см (5 см)",100 м2,1.555,38536.71,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1",59924.59
4,2.1-3103-18-1/1,"Устройство покрытий из асфальтобетонных смесей вручную, толщина 4 см (5 см)",100 м2,3.920,38536.71,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1",151063.91


### 3. Результат:
- **ID- id**-  Из справочника, приложение 3. ID
- **КПГЗ- kpgz** Выбранный пользователем или соответствующий выбранному шаблону ТЗ
- **СПГЗ- spgz** Один или несколько определенный сервисом. Може быть пустым, при этом не для каждой работы в смете должна (нужна) быть выявлена позиция СПГЗ
- **Объем- unit** Подтягивается из сметы: Кол-во единиц
- **Единица измерения- count** Подтягивается из сметы: Единица измерения
- **Цена за единицу ТРУ, руб- unit_price** *Рассчитываемое поле:* Стоимость за ТРУ, руб / Количество
- **Стоимость за ТРУ, руб- total** Подтягивается из сметы: ВСЕГО затрат, руб. При корректировке пол вручную должен быть произведен пересчет с предупреждением совпадения общей суммы по смете или возможность перераспределить остаток по всем работам в смете поровну или суммировать с выбранными позициями
- **Адрес** Вставляется при первоначальном выборе разбираемой сметы. При незаполненном поле осуществляется поиск адреса внутри сметы.

In [8]:
def get_key(df):

    # Откроем ключевые СПГЗ и КПГЗ с кодами-шифрами
    url = 'https://docs.google.com/spreadsheets/d/1a4n2yJjFjJQWJ26G5x-Ict43ALCLrP1S1UcUqAvFLHY/edit?usp=sharing'
    url.split('/')
    id = url.split('/')[5]
    spgz = pd.read_excel(f'https://docs.google.com/spreadsheets/d/{id}/export?format=xlsx')
       
    spgz = (
        spgz
        .drop(columns={'Unnamed: 3'})
        .rename(columns={'Шифр расценки и коды ресурсов': 'code',
                    'Наименование работ и затрат': 'work',
                    'Ед. изм.': 'count',
                    'Наименование СПГЗ': 'spgz',
                    'ID': 'id',
                    'КПГЗ': 'kpgz',
                    'Единицы измерения': 'unit'
        })
    )
    # Уберем дубликаты, но оставим одну первую строку.
    spgz = spgz.drop_duplicates(keep='first')

    # Объединим таблицы
    estimate_spgz = df[['code', 'work', 'unit_price', 'total', 'address']].merge(spgz, how='left', on=['code', 'work'])

    # Выделим столбцы для итоговой таблицы. Удалим дубликаты, если есть.
    df = (
        estimate_spgz[['id', 'kpgz', 'spgz', 'count', 'unit', 'unit_price', 'total', 'address']]
        .drop_duplicates(keep='first')
        .replace(np.nan, 0)
        .query('id != 0')
        .reset_index(drop=True)
    )
    return df

In [9]:
df = get_key(df)
print(df.shape)
df.head(10)

(18, 8)


,id,kpgz,spgz,count,unit,unit_price,total,address
0,91742866.0,02.03.03.11 ОБУСТРОЙСТВО ПОКРЫТИЙ И ЭЛЕМЕНТОВ СОПРЯЖЕНИЯ ТЕРРИТОРИЙ,Замена покрытия акрилового (хард) в рамках благоустройства территории,100 м2,Квадратный метр,0.0,0.0,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
1,91739067.0,02.06.05.04 РАБОТЫ РЕМОНТНО-ВОССТАНОВИТЕЛЬНЫЕ СВЯЗАННЫЕ С ПОКРЫТИЯМИ И ЭЛЕМЕНТАМИ СОПРЯЖЕНИЯ ТЕРРИТОРИЙ,Замена покрытия акрилового (хард) в рамках ремонтно-восстановительных работ на объектах благоустройства,100 м2,Квадратный метр,0.0,0.0,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
2,91738688.0,02.03.03.11 ОБУСТРОЙСТВО ПОКРЫТИЙ И ЭЛЕМЕНТОВ СОПРЯЖЕНИЯ ТЕРРИТОРИЙ,Замена покрытия из резиновой крошки в рамках благоустройства территории,100 м2,Квадратный метр,0.0,0.0,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
3,60046142.0,02.03.12 РАБОТЫ ПО БЛАГОУСТРОЙСТВУ ТЕРРИТОРИЙ ОБЪЕКТОВ СПОРТА,Замена покрытия из резиновой крошки в рамках благоустройства территорий объектов спорта,100 м2,Квадратный метр,0.0,0.0,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
4,46323952.0,"02.03.04 СТРОИТЕЛЬСТВО ПАРКОВ, МЕСТ ОТДЫХА И ДОСУГА",Устройство EPDM покрытия детской игровой площадки при благоустройстве парков,100 м2,Квадратный метр,0.0,0.0,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
5,46323931.0,"02.03.04 СТРОИТЕЛЬСТВО ПАРКОВ, МЕСТ ОТДЫХА И ДОСУГА",Устройство EPDM покрытия спортивной площадки при благоустройстве парков,100 м2,Квадратный метр,0.0,0.0,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
6,91741271.0,02.03.03.11 ОБУСТРОЙСТВО ПОКРЫТИЙ И ЭЛЕМЕНТОВ СОПРЯЖЕНИЯ ТЕРРИТОРИЙ,Устройство покрытия акрилового (хард) в рамках благоустройства территории,100 м2,Квадратный метр,0.0,0.0,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
7,91742563.0,02.06.05.04 РАБОТЫ РЕМОНТНО-ВОССТАНОВИТЕЛЬНЫЕ СВЯЗАННЫЕ С ПОКРЫТИЯМИ И ЭЛЕМЕНТАМИ СОПРЯЖЕНИЯ ТЕРРИТОРИЙ,Устройство покрытия акрилового (хард) в рамках ремонтно-восстановительных работ на объектах благоустройства,100 м2,Квадратный метр,0.0,0.0,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
8,91739664.0,02.03.03.11 ОБУСТРОЙСТВО ПОКРЫТИЙ И ЭЛЕМЕНТОВ СОПРЯЖЕНИЯ ТЕРРИТОРИЙ,Устройство покрытия из резиновой крошки в рамках благоустройства территории,100 м2,Квадратный метр,0.0,0.0,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
9,60046236.0,02.03.12 РАБОТЫ ПО БЛАГОУСТРОЙСТВУ ТЕРРИТОРИЙ ОБЪЕКТОВ СПОРТА,Устройство покрытия из резиновой крошки в рамках благоустройства территорий объектов спорта,100 м2,Квадратный метр,0.0,0.0,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"


### По этой смете получилось 10 ключевых работ.

In [10]:
# Сохраним результат
df.to_excel('estimate_final.xlsx', index=False)